<a href="https://colab.research.google.com/github/mandem296/Depression_detection_BERT_model/blob/main/BERT_Depression_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.8 MB/s 
     |████████████████████████████████| 7.6 MB 60.7 MB/s 
     |████████████████████████████████| 182 kB 80.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [29]:
df=pd.read_csv('/content/drive/MyDrive/project/Suicide_Detection.csv',encoding='latin-1',nrows=20000)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
df = df.loc[:, ['post_text','class']]

In [31]:
np.where(pd.isnull(df))

(array([10972, 10973, 10973, 10974, 10976, 10976, 10977, 10977, 10978,
        10978, 18067, 18068, 18068, 18069]),
 array([1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0]))

In [32]:
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [33]:
def assignNewLabels(label):
    if label == 'suicide':
        return 0
    else:
        return 1

In [34]:
df['label'] = df['class'].apply(assignNewLabels)

In [35]:
df

,post_text,class,label
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,0
1,Am I weird I don't get affected by compliments...,non-suicide,1
2,Finally 2020 is almost over... So I can never ...,non-suicide,1
3,i need helpjust help me im crying so hard,suicide,0
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide,0
...,...,...,...
19995,I was falsely convicted of rape and spent two ...,suicide,0
19996,Giving up hope soonI don't even know what to d...,suicide,0
19997,HurtI want to hurt myself today. I want to cau...,suicide,0
19998,ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh...,non-suicide,1


In [36]:
df['label'].value_counts()

1    10131
0     9860
Name: label, dtype: int64

In [37]:
df = df[['post_text','label']]
df = df[0:1000]
df.head()

,post_text,label
0,Ex Wife Threatening SuicideRecently I left my ...,0
1,Am I weird I don't get affected by compliments...,1
2,Finally 2020 is almost over... So I can never ...,1
3,i need helpjust help me im crying so hard,0
4,"Iâm so lostHello, my name is Adam (16) and I...",0


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [39]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased' ,num_labels=2)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_positio

In [40]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [41]:
model = model.to('cuda')

In [42]:
sample_data = ["I am sad","I am happy"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 6517, 102], [101, 1045, 2572, 3407, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [43]:
X = list(df["post_text"])
y = list(df["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [44]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [45]:
print(X_train_tokenized['input_ids'][0])

[101, 2034, 2028, 2000, 3786, 2033, 1999, 7532, 2176, 5222, 2019, 11809, 2400, 1037, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 1037, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 1037, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 1037, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 1037, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 2050, 29653, 1037, 29653, 2050, 29653, 2050, 29653, 29668, 29658, 2050, 29653, 2050, 29653, 2050, 29653, 2811, 2686, 2176, 2335, 2012, 1996, 2707, 1997, 2169, 2240, 2005, 2204, 4289, 3436, 1012, 6100, 19351, 2026, 2693, 2169, 2051, 1998, 2059, 2191, 6737, 1012, 1045, 1005, 2222, 7514, 2000, 2115, 7615, 1998, 2057, 2097, 2707, 2652, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [46]:
len(X_train),len(X_val)

(800, 200)

In [47]:
#fine tuning process
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [48]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [49]:
train_dataset[5]

{'input_ids': tensor([  101,  7987, 27225,  2054,  2003, 11235,  6069,  2079,  2005,  4268,
          1998, 14472,  2007,  4234,  2514,  2066, 11235,  2323,  3844,  1996,
          2816,  1016,  3134,  2077,  4234,  3338,  2000,  5547,  1996,  3891,
          2061,  4268,  2064,  3113,  2039,  2007,  2037,  2155,  2066,  3671,
          2302, 15366,  2055,  1996,  7865,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [50]:
def compute_metrics(p):
  print(type(p))
  pred, labels = p
  pred = np.argmax(pred, axis=1)

  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels, y_pred=pred)
  return {"accuracy": accuracy, "precision":precision, "recall":recall, "f1_score":f1}

In [51]:
#Defining the BERT trainer
args = TrainingArguments(
    output_dir = "output",
    num_train_epochs=2,
    per_device_train_batch_size=8
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 800
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 200
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.20972518920898436, metrics={'train_runtime': 148.358, 'train_samples_per_second': 10.785, 'train_steps_per_second': 1.348, 'total_flos': 420977688576000.0, 'train_loss': 0.20972518920898436, 'epoch': 2.0})

In [53]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.16976934671401978,
 'eval_accuracy': 0.955,
 'eval_precision': 0.9514563106796117,
 'eval_recall': 0.9607843137254902,
 'eval_f1_score': 0.9560975609756097,
 'eval_runtime': 6.6084,
 'eval_samples_per_second': 30.265,
 'eval_steps_per_second': 3.783,
 'epoch': 2.0}